In [1]:
import tensorflow as tf
import keras
from keras import layers
import numpy as np
import pandas as pd

latent_dim = 32
height = 32
width = 32
channels = 3

In [2]:
generator_input = keras.Input(shape=(latent_dim,))

x = layers.Dense(128 * 16 * 16)(generator_input)
x = layers.LeakyReLU()(x)
x = layers.Reshape((16, 16, 128))(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2DTranspose(256, 4, strides=2, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(256, 5, padding='same')(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(channels, 7, activation='tanh', padding='same')(x)
generator = keras.models.Model(generator_input, x)

generator.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32)]              0         
_________________________________________________________________
dense (Dense)                (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 32768)             0         
_________________________________________________________________
reshape (Reshape)            (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 32, 32, 256)       104883

## The GAN discriminator network

In [6]:
discriminator_input = layers.Input(shape=(height, width, channels))

x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)

discriminator = keras.models.Model(discriminator_input, x)
discriminator.summary()
# from tensorflow.keras.optimizers import RMSprop

discriminator_optimizer = tf.keras.optimizers.RMSprop(
    lr=0.0008,
    clipvalue=1.0,
    decay=1e-8)
discriminator.compile(optimizer=discriminator_optimizer,
                      loss='binary_crossentropy')

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_19 (LeakyReLU)   (None, 6, 6, 128)         0   

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## Adverserial Network

In [8]:
discriminator.trainable = False

gan_input = keras.Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output)

gan_optimizer = tf.keras.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)

gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## Implementing GAN training

In [10]:
import os

from keras.preprocessing import image
(x_train, y_train), (_, _) = keras.datasets.cifar10.load_data()

x_train = x_train[y_train.flatten() == 6]
x_train = x_train.reshape(
    (x_train.shape[0],) +
    (height, width, channels)).astype('float32') / 255.

iterations = 10000
batch_size = 20
save_dir = '/content/drive/MyDrive/weather_for'
start = 0

for step in range(iterations):
  random_latent_vectors = np.random.normal(size=(batch_size,
                                                 latent_dim))
  
  generated_images = generator.predict(random_latent_vectors)

  stop = start + batch_size

  real_images = x_train[start: stop]
  combined_images = np.concatenate([generated_images, real_images])
  labels = np.concatenate([np.ones((batch_size, 1)),
                           np.zeros((batch_size, 1))])
  
  labels += 0.05 * np.random.random(labels.shape)

  d_loss = discriminator.train_on_batch(combined_images, labels)

  random_latent_vectors = np.random.normal(size=(batch_size,
                                                 latent_dim))
  
  misleading_targets = np.zeros((batch_size, 1))

  a_loss = gan.train_on_batch(random_latent_vectors,
                              misleading_targets)

  start += batch_size
  if start > len(x_train) - batch_size:
    start = 0
  if step % 100 == 0:
    gan.save_weights('gan.h5')
    print('discriminator loss:', d_loss)
    print('adversarial loss:', a_loss)

    img = image.array_to_img(generated_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,
                          'generated_frog' + str(step) + '.png'))
    
    img = image.array_to_img(real_images[0] * 255., scale=False)
    img.save(os.path.join(save_dir,
                          'real_frog' + str(step) + '.png'))

discriminator loss: 6.083743095397949
adversarial loss: 13.869671821594238
discriminator loss: 0.6722233891487122
adversarial loss: 0.9225446581840515
discriminator loss: 0.7146993279457092
adversarial loss: 0.7924403548240662
discriminator loss: 0.6828715205192566
adversarial loss: 0.7984875440597534
discriminator loss: 0.7086002826690674
adversarial loss: 0.7146921157836914
discriminator loss: 0.6907554268836975
adversarial loss: 0.7349254488945007
discriminator loss: 0.7024229168891907
adversarial loss: 0.73198401927948
discriminator loss: 0.7012313008308411
adversarial loss: 0.7439326643943787
discriminator loss: 0.6938794255256653
adversarial loss: 0.7430118322372437
discriminator loss: 0.6934617757797241
adversarial loss: 0.7796405553817749
discriminator loss: 0.6813144683837891
adversarial loss: 0.8183659315109253
discriminator loss: 0.6969290971755981
adversarial loss: 0.7709927558898926
discriminator loss: 0.6880398988723755
adversarial loss: 0.7402913570404053
discriminator l

In [11]:
!cp '/content/gan.h5' -d '/content/drive/MyDrive/weather_for'